In [10]:
import numpy as np
import pandas as pd
import pandas as pd
from collections import Counter
import re

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [11]:
picks = pd.read_parquet('../../Data/2023_2025_pick_trades.parquet')
# Normalize franchise ownership
OWNER_REMAP = {
    "Weeks": "Gio"
}

picks = picks.copy()

picks["receiving_owner"] = picks["receiving_owner"].replace(OWNER_REMAP)
picks["giving_owner"] = picks["giving_owner"].replace(OWNER_REMAP)


In [12]:
# All managers
managers = pd.unique(
    pd.concat([picks['receiving_owner'], picks['giving_owner']])
)

def starting_picks_for_manager():
    picks = []

    # Initial picks (start of 2023)
    for year in [2024, 2025, 2026]:
        for rnd in range(1, 5):
            picks.append(f"{year}_{rnd}")

    # Added at start of 2024
    for rnd in range(1, 5):
        picks.append(f"2027_{rnd}")

    # Added at start of 2025 (not tradeable, but owned)
    for rnd in range(1, 5):
        picks.append(f"2028_{rnd}")

    return Counter(picks)

# Initialize inventory
inventory = {mgr: starting_picks_for_manager() for mgr in managers}


In [14]:
VALID_PICK_PATTERN = re.compile(r"^(2024|2025|2026|2027|2028)_[1-4]$")

In [15]:
picks_sorted = picks.sort_values("date")

for _, row in picks_sorted.iterrows():
    ro = row["receiving_owner"]
    go = row["giving_owner"]

    for pick in row["ro_picks_received"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[ro][pick] += 1

    for pick in row["ro_picks_gave"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[ro][pick] -= 1

    for pick in row["go_picks_received"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[go][pick] += 1

    for pick in row["go_picks_gave"]:
        if VALID_PICK_PATTERN.match(pick):
            inventory[go][pick] -= 1


In [17]:
rows = []

for owner, picks_counter in inventory.items():
    for pick, count in picks_counter.items():
        if count > 0:
            year, rnd = pick.split("_")
            rows.append({
                "owner": owner,
                "pick_year": int(year),
                "round": int(rnd),
                "count": count
            })

pick_tracker = (
    pd.DataFrame(rows)
      .sort_values(["owner", "pick_year", "round"])
      .reset_index(drop=True)
)


### Manual Adjustments

In [7]:
# # Jose: add 1 pick to 2026 Round 3
# pick_tracker.loc[
#     (pick_tracker["owner"] == "Jose") &
#     (pick_tracker["pick_year"] == 2026) &
#     (pick_tracker["round"] == 3),
#     "count"
# ] += 1


# # Bryan: remove 2 picks from 2026 Round 3
# pick_tracker.loc[
#     (pick_tracker["owner"] == "Bryan") &
#     (pick_tracker["pick_year"] == 2026) &
#     (pick_tracker["round"] == 3),
#     "count"
# ] -= 2


# # Drop any rows that now have 0 picks
# pick_tracker = (
#     pick_tracker
#     .loc[pick_tracker["count"] > 0]
#     .reset_index(drop=True)
# )


In [9]:
pick_tracker[pick_tracker['owner'] == 'Bryan']

,owner,pick_year,round,count
8,Bryan,2024,1,1
9,Bryan,2024,2,1
10,Bryan,2024,3,1
11,Bryan,2024,4,2
12,Bryan,2025,1,2
13,Bryan,2025,3,1
14,Bryan,2025,4,6
15,Bryan,2026,1,2
16,Bryan,2026,2,2
17,Bryan,2026,4,2


In [104]:
pick_tracker_future = pick_tracker[pick_tracker['pick_year'].isin([2026, 2027, 2028])]

### Visualization

In [105]:
ROUND_WEIGHTS = {
    1: 15.0,
    2: 8.0,
    3: 3.0,
    4: 1.0
}

# Copy to avoid mutation
df = pick_tracker.copy()

# Apply round weights
df["round_weight"] = df["round"].map(ROUND_WEIGHTS)

# Compute weighted pick value
df["weighted_value"] = df["count"] * df["round_weight"]

# Label used vs future picks
df["pick_status"] = df["pick_year"].apply(
    lambda x: "Used (2024–25)" if x <= 2025 else "Future (2026–28)"
)


In [106]:
plot_df = (
    df.groupby(["owner", "pick_year"], as_index=False)
      .agg(total_value=("weighted_value", "sum"))
)


In [ ]:
pivot_df = plot_df.pivot(
    index="owner",
    columns="pick_year",
    values="total_value"
).fillna(0)

# Sort managers by total pick value
pivot_df["total"] = pivot_df.sum(axis=1)
pivot_df = pivot_df.sort_values("total", ascending=False).drop(columns="total")

# Plot
ax = pivot_df.plot(
    kind="bar",
    stacked=True,
    figsize=(14, 7)
)

ax.set_title("Dynasty Pick Stockpile by Manager (Weighted by Round)", fontsize=14)
ax.set_xlabel("Manager")
ax.set_ylabel("Weighted Pick Value")

plt.legend(
    title="Pick Year",
    bbox_to_anchor=(1.02, 1),
    loc="upper left"
)

plt.tight_layout()
plt.show()


In [ ]:
future_df = df[df["pick_year"] >= 2026]

future_plot = (
    future_df.groupby("owner", as_index=False)
    .agg(future_value=("weighted_value", "sum"))
    .sort_values("future_value", ascending=False)
)

plt.figure(figsize=(12,6))
plt.bar(future_plot["owner"], future_plot["future_value"])
plt.title("Future Draft Capital (2026–2028, Weighted)")
plt.ylabel("Weighted Pick Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
heatmap_df = (
    df[df["pick_year"] >= 2026]
    .groupby(["owner", "round"], as_index=False)
    .agg(value=("weighted_value", "sum"))
    .pivot(index="owner", columns="round", values="value")
    .fillna(0)
)
heatmap_df

In [110]:
time_series = (
    df.groupby(["owner", "pick_year"], as_index=False)
      .agg(total_value=("weighted_value", "sum"))
)

all_owners = time_series["owner"].unique()
all_years = sorted(time_series["pick_year"].unique())

full_index = pd.MultiIndex.from_product(
    [all_owners, all_years],
    names=["owner", "pick_year"]
)

time_series_complete = (
    time_series
    .set_index(["owner", "pick_year"])
    .reindex(full_index)
    .fillna(0)
    .reset_index()
)

In [ ]:
plt.figure(figsize=(14, 7))

for owner, owner_df in time_series_complete.groupby("owner"):
    owner_df = owner_df.sort_values("pick_year")
    plt.plot(
        owner_df["pick_year"],
        owner_df["total_value"],
        marker="o",
        linewidth=2,
        alpha=0.85,
        label=owner
    )

# Vertical cutoff line at 2025
plt.axvline(
    x=2025,
    linestyle="--",
    linewidth=2,
    color="red",
    alpha=0.8
)

plt.text(
    2025 + 0.05,
    plt.ylim()[1] * 0.95,
    "Picks Used Through 2025",
    color="red",
    fontsize=10
)

plt.title("Dynasty Draft Capital Over Time (Weighted by Round)", fontsize=14)
plt.xlabel("Draft Year")
plt.ylabel("Weighted Pick Value")

plt.xticks(sorted(time_series_complete["pick_year"].unique()))
plt.legend(
    title="Manager",
    bbox_to_anchor=(1.02, 1),
    loc="upper left"
)
plt.axvspan(2024,2025, color = 'lightgray')

plt.tight_layout()
plt.show()
